In [ ]:
using ContGridModML
using CairoMakie
using Statistics
using Printf
using LaTeXStrings

This will learn the susceptances from the stable solutions of 48 different dispatches.
The test set performance is calculated on 12 different dispatches.
The power is distributed using the heat equation procedure described in *Pagnier et al., IEEE Access 10, 65118 (2022)*. 

In [ ]:
# Set the seed to exactly reproduce the results of the paper (this only changes the intial values)
sol = learn_susceptances_dates()

In [ ]:
save_sol("static.h5", sol)

Alternatively just load the provided solution file

In [ ]:
sol = load_sol("static.h5");

We can plot the learned values using the ContGridModML functionality

In [ ]:
nodal_plot(sol.model, :bx, logarithmic = false, cbar_args = Dict(:label => L"b_x"))

In [ ]:
nodal_plot(sol.model, :by, logarithmic = false, cbar_args = Dict(:label => L"b_y"))

Let's look at the trainings performance. We start by plotting the trainings error during the training.

In [ ]:
n_epochs = size(sol.losses, 1)
n_batch = size(sol.losses, 2)
f = Figure()
ax = Axis(f[1, 1], xlabel = "Epoch", ylabel = "Loss")
x = range(1 - 1 / n_batch, n_epochs + 1 / n_batch, n_batch * n_epochs)
y = vcat(sol.losses'...)
lines!(ax, x, y, label = "Batch Loss")
x2 = range(1, n_epochs, n_epochs)
y2 = vcat(mean(sol.losses, dims = 2)...)
lines!(ax, x2, y2, label = "Epoch Loss")
axislegend()
f

Next, we plot the predicted angles vs. the ground truth

In [ ]:
f = Figure(resolution = (1200, 3200))
for i in 1:48
    mi = min(minimum(sol.t_train[:, i]), minimum(sol.train_pred[:, i]))
    ma = max(maximum(sol.t_train[:, i]), maximum(sol.train_pred[:, i]))
    mi -= 0.05 * (ma - mi)
    ma += 0.05 * (ma - mi)
    ax = Axis(f[(i - 1) ÷ 4 + 1, mod(i - 1, 4) + 1],
        aspect = DataAspect(),
        limits = (mi, ma, mi, ma))
    scatter!(ax, sol.t_train[:, i], sol.train_pred[:, i], strokewidth = 1)
    ablines!(ax, 0, 1, color = :red)
    text!(ax,
        0.05,
        0.95,
        text = @sprintf("Loss = %.2e", sol.train_losses[i]),
        space = :relative,
        align = (:left, :top))
end
f[0, :] = Label(f, @sprintf("Mean loss = %.2e", mean(sol.train_losses)), fontsize = 25)
f[:, 0] = Label(f, L"\theta_\mathrm{pred}", rotation = π / 2, fontsize = 25)
f[13, :] = Label(f, L"\theta_\mathrm{true}", fontsize = 25)
f

In [ ]:
f = Figure(resolution = (1200, 1700))
for i in 1:12
    mi = min(minimum(sol.t_test[:, i]), minimum(sol.test_pred[:, i]))
    ma = max(maximum(sol.t_test[:, i]), maximum(sol.test_pred[:, i]))
    mi -= 0.05 * (ma - mi)
    ma += 0.05 * (ma - mi)
    ax = Axis(f[(i - 1) ÷ 3 + 1, mod(i - 1, 3) + 1],
        aspect = DataAspect(),
        limits = (mi, ma, mi, ma))
    scatter!(ax, sol.t_test[:, i], sol.test_pred[:, i], strokewidth = 1)
    ablines!(ax, 0, 1, color = :red)
    text!(ax,
        0.05,
        0.95,
        text = @sprintf("Loss = %.2e", sol.test_losses[i]),
        space = :relative,
        align = (:left, :top))
end
f[0, :] = Label(f, @sprintf("Mean loss = %.2e", mean(sol.test_losses)), fontsize = 25)
f[:, 0] = Label(f, L"\theta_\mathrm{pred}", rotation = π / 2, fontsize = 25)
f[5, :] = Label(f, L"\theta_\mathrm{true}", fontsize = 25)
f

In [ ]:
seed = 1709
sol = learn_dynamical_parameters(n_train = 4, n_test = 1, n_epochs = 10)

In [ ]:
sol = load_sol("dynamic.h5");

In [ ]:
save_sol("dynamic.h5", sol)

In [ ]:
f = Figure()
for i in 1:4
    ax = Axis(f[(i - 1) ÷ 2 + 1, mod(i - 1, 2) + 1])
    lines!(ax, sol.losses[:, i])
end
f